<a href="https://colab.research.google.com/github/aninsung/Machine-Learning-Programming/blob/main/4%EC%A3%BC%EC%B0%A8_mhealth_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (오늘 실습과제)

Mhealth dataset으로 인간행동 분류하기

일단 데이터 읽어서 subject 번호가 컬럼면에 포함된 통합 data frame 만들고 CNN, LSTM, CNN-LSTM 해보기.

1. LLM(sLLM)을 제외한 어떤 모델을 써도 가능. 단 라이브러리 import 불가. 모델의 풀 소스가 있어야 함.

 2. 10명을 1+2, 3+4, 4+5,.... 5개그룹으로 나눠 2명씩 테스트 데이터로 하면서 cross validation.

 3. 데이터 증대는 안됨.

4. 오직 모델로만 성능 향상

5. 상위 30%씩 차등.

6. 성적에 반영

7. 11월15일까지. Github게시

교차 평가로 평균 F1 기준.

8. 본인이 완벽하게 소스 및 모델을 설명할 수 있어야 함. GPT를 쓰던 뭘 쓰던 그건 상관안함. 재현성이 보장되어야 함.
--------------------------------------------------------------------------
--------------------------------------------------------------------------

## 데이터설명
📌 Dataset Summary

Subjects: 10명 (다양한 프로필의 자원자)

Activities: 12개

Sensors: 3개 (가슴, 오른쪽 손목, 왼쪽 발목 → elastic strap으로 고정)

측정 정보:

Acceleration (가속도)

Gyroscope (회전율)

Magnetometer (자기장 방향)

Chest sensor: 2-lead ECG 추가 (활동 인식에는 사용 안 함, 향후 연구용)

Sampling Rate: 50 Hz

환경: Out-of-lab, 제약 없이 자유롭게 활동 수행 (단, 최선을 다해 실행)

📌 Activity Set (12 classes)

정적 활동: Standing still, Sitting, Lying down

일상/이동 활동: Walking, Climbing stairs, Cycling

운동/역동적 활동: Jogging, Running, Jump front & back

반복 동작: Waist bends, Arm elevation, Knees bending

📌 특징

ECG 포함: 운동 중 심전도 기록 → 부정맥, 심박수 모니터링, 운동 효과 분석 가능

다양성 확보:

동작 강도 차이 (예: Sitting vs. Running)

속도 차이 (예: Walking vs. Jogging)

다른 신체 부위 동작 (예: Arm elevation vs. Knees bending)

실생활 반영: 실제 환경에서 수행되어 일반화 성능 높음

In [23]:
import pandas as pd
import glob
import numpy as np

# 1. 파일 리스트
files = sorted(glob.glob("/content/drive/MyDrive/Colab Notebooks/mhealth+dataset/MHEALTHDATASET/mHealth_subject*.log"))

# 2. 컬럼 정의
columns = [
    "chest_acc_x","chest_acc_y","chest_acc_z",
    "ecg_1","ecg_2",
    "chest_gyro_x","chest_gyro_y","chest_gyro_z",
    "chest_mag_x","chest_mag_y","chest_mag_z",
    "ankle_acc_x","ankle_acc_y","ankle_acc_z",
    "ankle_gyro_x","ankle_gyro_y","ankle_gyro_z",
    "ankle_mag_x","ankle_mag_y","ankle_mag_z",
    "wrist_acc_x","wrist_acc_y","wrist_acc_z",
    "label"
]

# 3. 각 파일 읽어서 subject_id 추가
dfs = []
for i, file in enumerate(files, start=1):
    df = pd.read_csv(file, sep="\s+", header=None)
    df.columns = columns
    df["subject_id"] = i
    dfs.append(df)

# 4. 통합 DataFrame
data = pd.concat(dfs, ignore_index=True)


<>:24: SyntaxWarning: invalid escape sequence '\s'
<>:24: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-4155912323.py:24: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file, sep="\s+", header=None)


In [24]:
def create_windows(df, frame_size=100, hop_size=50):
    X, y = [], []
    data_values = df.drop(["label", "subject_id"], axis=1).values
    labels = df["label"].values
    for i in range(0, len(data_values)-frame_size, hop_size):
        X.append(data_values[i:i+frame_size])
        y.append(labels[i+frame_size-1])  # 마지막 시점 label
    return np.array(X), np.array(y)


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM

n_classes = data["label"].nunique()
n_features = data.drop(["label","subject_id"],axis=1).shape[1]
timesteps = 100  # frame_size

# CNN 모델
def build_cnn():
    model = Sequential([
        Conv1D(64, 3, activation="relu", input_shape=(timesteps, n_features)),
        MaxPooling1D(2),
        Flatten(),
        Dense(64, activation="relu"),
        Dense(n_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# LSTM 모델
def build_lstm():
    model = Sequential([
        LSTM(64, input_shape=(timesteps, n_features)),
        Dense(64, activation="relu"),
        Dense(n_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# CNN-LSTM 모델
def build_cnn_lstm():
    model = Sequential([
        Conv1D(64,3, activation="relu", input_shape=(timesteps, n_features)),
        MaxPooling1D(2),
        LSTM(64),
        Dense(64, activation="relu"),
        Dense(n_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


In [26]:
subjects = np.arange(1,11)
folds = [(subjects[i], subjects[i+1]) for i in range(0,10,2)]

for fold_idx, test_subjects in enumerate(folds, start=1):
    # Train/Test 분리
    test_data = data[data["subject_id"].isin(test_subjects)]
    train_data = data[~data["subject_id"].isin(test_subjects)]

    # 윈도우 생성
    X_train, y_train = create_windows(train_data, frame_size=timesteps, hop_size=50)
    X_test, y_test = create_windows(test_data, frame_size=timesteps, hop_size=50)

    print(f"\n=== Fold {fold_idx}: Test subjects {test_subjects} ===")
    print("Samples:", X_train.shape[0], "(train)", X_test.shape[0], "(test)")

    # 모델 선택
    model = build_cnn()  # CNN, LSTM, CNN-LSTM 중 선택
    # model = build_lstm()
    # model = build_cnn_lstm()

    # 학습
    model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)

    # 평가
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {acc:.4f}")



=== Fold 1: Test subjects (np.int64(1), np.int64(2)) ===
Samples: 19122 (train) 5190 (test)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6453 - loss: 3.0611
Epoch 2/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.7875 - loss: 0.5067
Epoch 3/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8191 - loss: 0.3953
Epoch 4/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8415 - loss: 0.3442
Epoch 5/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.8550 - loss: 0.3150
Test Accuracy: 0.7778

=== Fold 2: Test subjects (np.int64(3), np.int64(4)) ===
Samples: 19260 (train) 5052 (test)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6586 - loss: 2.6578
Epoch 2/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.7874 - loss: 0.5578
Epoch 3/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.8232 - loss: 0.3877
Epoch 4/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.8496 - loss: 0.3315
Epoch 5/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.8622 - loss: 0.3079
Test Accuracy: 0.7928

=== Fold 3: Test subjects (np.int64(5), np.int64(6)) ===
Samples: 19583 (train) 4729 (test)


...

Epoch 1/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.6512 - loss: 3.2530
Epoch 2/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.8119 - loss: 0.5056
Epoch 3/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.8365 - loss: 0.3808
Epoch 4/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.8617 - loss: 0.3276
Epoch 5/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.8704 - loss: 0.2946
Test Accuracy: 0.7866

=== Fold 4: Test subjects (np.int64(7), np.int64(8)) ===
Samples: 20258 (train) 4054 (test)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6525 - loss: 3.2900
Epoch 2/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.7612 - loss: 0.5983
Epoch 3/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.8002 - loss: 0.4582
Epoch 4/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.8143 - loss: 0.4106
Epoch 5/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8342 - loss: 0.3710
Test Accuracy: 0.7129

=== Fold 5: Test subjects (np.int64(9), np.int64(10)) ===
Samples: 19030 (train) 5282 (test)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.6306 - loss: 4.5614
Epoch 2/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.7629 - loss: 0.6360
Epoch 3/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.8052 - loss: 0.4613
Epoch 4/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8350 - loss: 0.3817
Epoch 5/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8459 - loss: 0.3379
Test Accuracy: 0.7741
